In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torchvision.transforms import v2
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm

transform = torch.nn.Sequential(
    v2.Resize(256),
    v2.RandomRotation(30),
    v2.RandomCrop(64),
    v2.RandomHorizontalFlip(),
    v2.ToTensor(),
    v2.Normalize(
        [0.5, 0.5, 0.5], 
        [0.5, 0.5, 0.5]))

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

In [ ]:
import requests

r = requests.get("http://localhost:8080/isalive")
print(r)
# <Response [200]>

In [ ]:
# Testing on local container

import base64

with open("./test_image.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

r = requests.post("http://localhost:8080/predict", json={
    "instances": [{"image": str(encoded_string)}]
})
print(r)
# <Response [200]>

r_json = r.json()
print(r_json)

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + "/psychic-surf-404204-7c8e524715fa.json"

In [ ]:
from typing import Dict, List, Union
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if type(instances) == list else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    return response.predictions

In [ ]:
preds = predict_custom_trained_model_sample(
    project="601592189510",
    endpoint_id="6377996472868143104",
    location="us-central1",
    instances={ "image": str(encoded_string) }
)

print(preds)


In [ ]:
# Testing on Google Cloud

with open("./test_image.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

r = requests.post("https://us-central1-aiplatform.googleapis.com/v1/projects/601592189510/locations/us-central1/endpoints/6377996472868143104:predict", json={
    "instances": [{"image": str(encoded_string)}]
})
print(r)
# <Response [200]>

r_json = r.json()
print(r_json)